Compare and visualize performance of a domestic asset vs foreign
===

In [1]:
import eikon as ek
import pandas as pd
import plotly.offline as offline

In [2]:
# eikon id is generated from Eiokon API proxy
ek.set_app_id('your_app_id_here')
# set notebook mode to allow charts to be generated inline
offline.offline.init_notebook_mode(connected=True)

Define variables


In [3]:
# the two instruments whose performance is to be compared
instr1 = 'TD.TO';
instr2 = 'BARC.L'
# the date range for comparison
start_date = '2015-01-01'
end_date = '2017-01-01'

Get the primary currency in which these instruments trade


In [4]:
data,err = ek.get_data([instr1, instr2], "TR.CompanyMarketCap.Currency")
# display the trading currencies
data.head()

,Instrument,Currency
0,TD.TO,CAD
1,BARC.L,GBP


Create a currency cross rate RIC with this information, and get timeseries data for this pair

In [5]:
# cross rate ric = curr1curr2=X. It is a calculated metric
crossRIC = data['Currency'][1] + data['Currency'][0] + '=X'
curr = ek.get_timeseries([crossRIC], fields='CLOSE', start_date=start_date, end_date=end_date)
curr.head()

GBPCAD=X,CLOSE
Date,
2015-01-01,1.8054
2015-01-02,1.8057
2015-01-05,1.7935
2015-01-06,1.7930
2015-01-07,1.7849


Get timeseries data for both instruments in the date range

In [6]:
perf1 = ek.get_timeseries(instr1, fields='CLOSE', start_date=start_date, end_date=end_date)
perf2 = ek.get_timeseries(instr2, fields='CLOSE', start_date=start_date, end_date=end_date)

Scale the performance data for foreign asset with curreny performance

In [7]:
# create a single DataFrame with both data series (useful if using cufflinks to plot instead of plotly)
perf = pd.concat([perf1, perf2 * curr], axis=1)
perf.columns = [instr1, instr2]
# drop invalid entries from dataframe
perf = perf.dropna()
perf.head()

,TD.TO,BARC.L
Date,,
2015-01-02,55.22,439.055955
2015-01-05,53.94,420.934450
2015-01-06,53.06,413.017550
2015-01-07,52.89,411.597940
2015-01-08,53.23,423.039430


Normalize the data

In [8]:
# rescale both timeseries to 100% at start date
perf_norm = perf * 100 / (perf[instr1][0], perf[instr2][0])
perf_norm.head()

,TD.TO,BARC.L
Date,,
2015-01-02,100.000000,100.000000
2015-01-05,97.681999,95.872621
2015-01-06,96.088374,94.069456
2015-01-07,95.780514,93.746124
2015-01-08,96.396233,96.352054


Plot this normalized dataseries in a chart

In [9]:
# using plotly for charting
offline.iplot([{
    'x': perf_norm.index,
    'y': perf_norm[col],
    'name': col
}  for col in perf_norm.columns])

# if using cufflinks, dataframe can be charted directly using command: perf_norm.iplot()